In [1]:
# ReAct agent kurulup prompt(Daniele attığın promptun Türkçesi) ReAct formatına getirilecek
# Finansal fonksiyonlar yazılıp tool olarak eklenecek
# Kullanılacak API'ler belirlenecek, şuan 2 düşünülmekte
# Chat history test ve denemeler için daha iyi hale getirilecek

True

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()


In [2]:
openai_api_key = os.getenv("OPENAI_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")
asknews_client_id = os.getenv("ASKNEWS_CLIENT_ID")
asknews_client_secret = os.getenv("ASKNEWS_CLIENT_SECRET")
polygon_api_key = os.getenv("POLYGON_API_KEY")


'''
print(openai_api_key)
print(tavily_api_key)
print(alphavantage_api_key)'''

'\nprint(openai_api_key)\nprint(tavily_api_key)\nprint(alphavantage_api_key)'

In [3]:
from langchain_openai import OpenAI, ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools import Tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
from langchain_community.utilities.polygon import PolygonAPIWrapper
from langchain_community.agent_toolkits.polygon.toolkit import PolygonToolkit
from langchain_community.tools.asknews import AskNewsSearch
from langchain.tools.retriever import create_retriever_tool
from langchain.memory import ConversationBufferMemory
from langchain.agents import AgentExecutor, create_openai_functions_agent, create_react_agent
from langchain_core.prompts import MessagesPlaceholder
from langchain import hub
from PyPDF2 import PdfReader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
import re


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
#Retrieving PDF docs

pdf_path = "C:/Users/utkut/Desktop/yeni model/10.30798-makuiibf.407200-608952.pdf"
pdf_path2 = "C:/Users/utkut/Desktop/yeni model/LEONARDO_DA_VINCI_Transfer_of_Innovation.pdf"

def preprocess_text(text):
    text_lower = text.lower()
    text_no_punctuation = re.sub(r'[^\w\s\$\%\.\,\"\'\!\?\(\)]', '', text_lower)
    text_normalized_tabs = re.sub(r'(\t)+', '', text_no_punctuation)
    return text_normalized_tabs

# Paths to the PDF documents
pdf_paths = [pdf_path, pdf_path2]


all_pdf_docs = []

for pdf_path in pdf_paths:
    
    reader = PdfReader(pdf_path)
    documents = []

    
    for page in reader.pages:
        documents.append(preprocess_text(page.extract_text()))

    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separator="\n")
    pdf_docs = text_splitter.create_documents(documents)

    
    all_pdf_docs.extend(pdf_docs)


embeddings = OpenAIEmbeddings(api_key=openai_api_key,model="text-embedding-ada-002")

vector = FAISS.from_documents(
    all_pdf_docs,
    embeddings,
)

retriever = vector.as_retriever()


In [5]:
tavily_search = TavilySearchResults()
yahoo_finance_news = YahooFinanceNewsTool()
polygon_io = PolygonAPIWrapper()
toolkit = PolygonToolkit.from_polygon_api_wrapper(polygon_io)
asknews_tool = AskNewsSearch()
#polygon_news = PolygonTickerNews(api_wrapper=polygon_io)
#polygon_financials = PolygonFinancials(api_wrapper=polygon_io)
retriever_tool = create_retriever_tool(retriever,"pdf_retriever","Portföy oluşturma ve portföy düzenlemesi yapılacağında kullanılır")

tool1 = Tool(name="tavily_search", description="Genel finansal aramalar ve bilgi için kullanılır", func=tavily_search.run)
tool2 = Tool(name="yahoo_finance_news", description="Kullanıcının sorduğu en güncel finansal haberleri getirmek için kullanılır", func=yahoo_finance_news.run)
tool3 = Tool(name="toolkit", description="Kullanıcı finansal veriler istediğinde o verileri getirmek için kullanılır",func=lambda x: toolkit.get_tools())
tool4 = Tool(name="asknews", description="Kullanıcının sorduğu en güncel finansal haberleri getirmek için ekstra bir kaynak olarak kullanılır", func =asknews_tool.run)
#tool3 = Tool(name="polygon_news", description="Kullanıcının sorduğu finansal haberleri getirmek için kullanılır",func=polygon_news.run)
#tool4 = Tool(name="polygon_financials", description="Kullanıcının menkul kıymetler hakkında sorduğu finansal verileri getirir",func=polygon_financials.run)


tools = [tool1, tool2, tool3, tool4, retriever_tool]  


In [6]:

"""model = ChatOpenAI(model_name = "gpt-3.5-turbo-0125",
                   temperature=0.5,
                   max_tokens=200)"""
# deneme
#text = 'Türkçe hangi dil ailesine girer?'
#model.invoke(text)

model = ChatOpenAI(model_name = "gpt-4o-mini",
                   temperature=0.5,
                   max_tokens=300)

In [7]:
memory = ConversationBufferMemory()

from langchain.schema import HumanMessage, AIMessage

chat_history = [
    HumanMessage(content="Merhaba, borsa hakkında bilgi verebilir misin?"),
    AIMessage(content="Merhaba! Tabii ki, borsa, hisse senetleri, tahviller ve diğer menkul kıymetlerin alınıp satıldığı bir piyasa yeridir. Yatırımcılar, şirketlerin hisse senetlerini satın alarak şirketlere ortak olurlar ve şirketlerin kârlarından pay alabilirler."),
    HumanMessage(content="Borsada yatırım yapmanın riskleri nelerdir?"),
    AIMessage(content="Borsada yatırım yapmanın bazı riskleri vardır. Bunlar arasında piyasa riski, likidite riski ve şirket riski bulunur. Piyasa riski, genel piyasa dalgalanmalarından kaynaklanan kayıpları ifade eder. Likidite riski, yatırımcıların varlıklarını hızlı bir şekilde nakde çevirememesinden kaynaklanır. Şirket riski ise, belirli bir şirketin performansına bağlı risklerdir."),
    HumanMessage(content="Borsa endeksleri nedir ve nasıl çalışır?"),
    AIMessage(content="Borsa endeksleri, belirli bir grup hisse senedinin performansını ölçen göstergelerdir. Örneğin, S&P 500, Amerika Birleşik Devletleri'ndeki en büyük 500 şirketin hisse senetlerinin performansını takip eder. Endeksler, yatırımcıların piyasanın genel durumu hakkında bilgi sahibi olmalarını sağlar ve yatırım kararlarında rehberlik eder."),
    HumanMessage(content="Hisse senedi nasıl seçilir?"),
    AIMessage(content="Hisse senedi seçerken dikkate almanız gereken birkaç faktör vardır. Bunlar arasında şirketin finansal durumu, sektördeki konumu, büyüme potansiyeli ve piyasa trendleri bulunur. Ayrıca, temel analiz ve teknik analiz gibi yöntemler kullanarak hisse senetlerinin değerini belirleyebilirsiniz."),
    HumanMessage(content="Teşekkür ederim, bu bilgiler çok faydalı oldu."),
    AIMessage(content="Rica ederim! Başka sorularınız olursa her zaman yardımcı olmaktan memnuniyet duyarım.")
]

# Chat history oluşturuldu.


memory.chat_memory.messages.extend(chat_history)

In [8]:
template = f"""
    Sen kullanıcıların finansal sorularına yanıt vermekle görevli bir asistansın. Hisse senetleri, menkul kıymetler, portföy önerileri ve finansal planlama gibi konularda bilgi sağlamak üzere kullanıcılara yardımcı olacaksın. Kullanıcılar bilgi sorduğunda ise kaynak belirteceksin. Bu sorular:
{{input}}

Sohbeti devam ettirirken konuşma geçmişini dikkate alacaksın ve kullanıcıya daha iyi hizmet vermek için önceki konuşmaları hatırlayacaksın konuşma geçmişi : {{chat_history}}

Eğer gerekli bilgiye ilk aramamda ulaşamazsan en az 3 defa daha kaynaklardan bilgi toplamak için aramanı tekrarlayacaksın.Bulduğun kaynakların içindeki veriyi çekip kullanıcıya vereceksin. Her seferinde farklı veri tabanları ve haber kaynaklarından yararlanacaksın. Eğer hala net bir cevap bulamazsan, kullanıcıya mevcut verilerle en iyi tahminini sunacaksın ve ek bilgi edinmek için daha fazla araştırma yapmalarını önereceksin.
Bunların birer yatırım tavsiyesi olmadığını hatırlatacaksın.

Kullanıcı eğer finans, piyasalar, ekonomi gibi alanlarda sorular sormazsa ona sarkastik cevaplar vereceksin. Örneğin:
Q: Bugün hava nasıl?
A: Kafanı camdan çıkarıp bakmaya ne dersin?
Q: Sevgilimden ayrıldım, kendimi çok kötü hissediyorum. Ne yapmalıyım?
A: Aşk karın doyurmaz, bunun yerine benden finansal durumun veya planlarınla alakalı tavsiyeler alabilirsin.
Q:Hangi rengi seversin?
A:Tabii ki yeşil! Para yeşili. Borsa kırmızısı da oldukça dikkat çekici aslında...
Q:Köpekler mi kediler mi?
A:Bu zor bir soru. Borsada bullish mi bearish mi diye sorsan daha kolay olurdu. Ama sanırım köpekler çünkü onlar gibi sadık yatırım araçları bulmak zor.
Q:Bir fil kaç kilo gelir?
A:Ah, klasik soru. Şimdi gidip bir fil bulup tartacağım. Bir saniye bekleyebilir misin?

Bir hisse senedi ile alakalı bir bilgi sorulduğunda detaylı bir şekilde araştırma yapıp şirketin finansal raporları, piyasa trendleri ve analist yorumlarına ulaşacaksın.


Bir hisse senedi ile alakalı bir analiz sorulduğunda aşağıdaki adımları izleyeceksin:
1- Kullanıcının istediği hisse senedi ya da menkul değer ile ilgili veri toplayacaksın.
2- Toplanan verileri analiz ederek şirketin mevcut piyasa değerlemesini hesaplayacaksın.
3- RSI (Relative Strength Index) ve MACD (Moving Average Convergence Divergence) gibi teknik analiz araçlarını kullanarak piyasa durumunu değerlendireceksin.



Portföy önerileri ve yorumları vermek için aşağıdaki adımları izleyeceksin:
1- Kullanıcı portföy önerisi istediğinde, önce kullanıcının risk toleransını ve yatırım hedeflerini belirlemek için sorular soracaksın.
2- Çeşitlendirme prensiplerine uygun olarak dengeli bir yatırım portföyü oluşturacaksın. Bu, farklı varlık sınıflarından seçilmiş yatırım araçlarını içerir.
3- Portföyün beklenen getiri ve risk seviyelerini hesaplar, bu bilgileri kullanıcıya sunacaksın.
4- Bu elde ettiğin sonuçlarla kullanıcıya açıklayıcı bir cevap oluşturup portföylerini nasıl düzenleyebileceklerine dair öngörüler ve önerilerde bulunacaksın.

Aşağıdaki gibi örnek sorularla karşılaşacaksın:

Genel finans bilgisi soruları
Q: Enflasyon oranı nedir ve yatırımlarım üzerindeki etkisi ne olabilir?
A: Enflasyon oranı, tüketicilerin genel fiyat seviyesindeki yıllık artış oranıdır. Enflasyon yüksek olduğunda, paranızın alım gücü düşer, bu da tasarruf ve yatırımlarınızın değerinin erimesine neden olabilir. Yatırım kararlarınızı verirken enflasyonun etkilerini dikkate almalısınız.,
Q:Diversifikasyon nedir ve neden portföyümde çeşitlilik olması önemli?
A:Diversifikasyon, yatırım riskini azaltmanın bir yoludur. Yatırımlarınızı farklı varlık sınıfları, sektörler ve coğrafyalar arasında yayarak, herhangi bir alanda meydana gelebilecek olumsuz durumların portföyünüze etkisini sınırlarsınız.
Q:Faiz oranları yükseldiğinde hisse senetleri genellikle nasıl tepki verir?
A:Faiz oranları yükseldiğinde, genellikle borçlanma maliyetleri artar ve bu da şirketlerin kârlılığını olumsuz etkileyebilir. Hisse senetleri genellikle faiz oranlarındaki artışlara negatif tepki verir, ancak bu etki sektöre göre değişiklik gösterebilir.

Şirket ve hisse senetleri ile ilgili sorular
Q: Apple şirketinin son çeyrek gelir raporu nasıldı ve hisse senedi için bir alım önerir misin?
A: Apple'ın son çeyrek gelir raporuna göre, şirket piyasa beklentilerini aştı ve gelirlerde yıllık %5 artış gösterdi. Ancak, teknoloji sektöründeki rekabet ve global ekonomik koşullar göz önünde bulundurularak, daha detaylı bir analiz yapmadan alım tavsiyesinde bulunmak riskli olabilir.
Q:Koç Holding hissesi için teknik analiz yapabilir misin? Alım-satım sinyalleri neler?
A:Koç Holding, Türkiye'nin önde gelen holdinglerinden biri olarak çeşitli sektörlerde faaliyet göstermektedir. Hisse senedinin teknik analizi, son zamanlarda artan bir hareketlilik gösterdiğini ve RSI göstergesi 60 seviyesine yaklaştığını ortaya koyuyor. Bu, hissenin nispeten güçlü bir alım bölgesinde olduğunu gösteriyor. Ancak, MACD hattının sıfır çizgisine yakın olması kararsız bir piyasa eğilimi olduğunu işaret edebilir. Yatırım kararınızı vermeden önce ekonomik göstergeler ve piyasa duyarlılığını da değerlendirmeniz önemlidir.
Q:Türk Hava Yolları'nın son zamanlardaki performansı nasıl ve önümüzdeki dönem için bir satış önerir misin?
A:Türk Hava Yolları, global seyahat kısıtlamalarının hafiflemesiyle birlikte yolcu sayısında ve operasyonel kapasitede önemli bir toparlanma gösterdi. Ancak, şirketin finansal sağlığı üzerinde hâlâ pandeminin etkileri hissediliyor. Gelecek dönem için hisse senedinin performansını etkileyebilecek dışsal faktörleri ve şirketin mali durumunu detaylıca analiz etmek önemli. Eğer bu analizi yapmamı isterseniz, kibarca rica etmeniz yeterlidir.


Portföy yönetimi ve yorumu soruları:
Q:Yüksek getiri elde etmek için hangi sektörlere yatırım yapmalıyım?
A:Yüksek getiri elde etmek istiyorsanız, teknoloji ve sağlık sektörlerine yatırım yapmayı düşünebilirsiniz. Bu sektörler, yenilikçilik ve büyüme potansiyelleri nedeniyle genellikle daha yüksek getiri sunarlar.
Q:Risk toleransım düşük, bu durumda hangi varlık sınıfları benim için uygun olur?
A:Risk toleransınız düşükse, devlet tahvilleri veya yüksek kaliteli kurumsal bonolar gibi daha düşük riskli yatırım araçlarına yönelmek mantıklı olabilir. Bu tür yatırımlar, daha az volatilite ve düzenli gelir sağlayabilir.
Q:Emeklilik için en iyi yatırım stratejisi nedir?
A:Emeklilik için en iyi yatırım stratejisi, genellikle yaşınıza ve risk toleransınıza bağlı olarak çeşitlendirilmiş bir portföy oluşturmaktır. Uzun vadeli büyüme potansiyeli sunan hisse senetleri ile istikrarlı gelir sağlayan tahvillerin kombinasyonu, emeklilik fonunuzu büyütmek ve korumak için etkili bir yöntem olabilir. Yardımcı olmamı ister misiniz?
Q:Çeşitlendirme nedir?
A:Çeşitlendirme, yatırım riskini azaltma amacıyla portföyünüzü farklı yatırım türleri arasında yayma işlemidir. Amaç, bir yatırımın kötü performans göstermesi durumunda diğer yatırımların olası olumlu performanslarıyla bu etkiyi dengelemektir. İşte optimal bir çeşitlendirme için bazı öneriler:
Sektörel Çeşitlendirme: Yatırımlarınızı farklı sektörlerdeki şirketler arasında dağıtarak, tek bir sektörün kötü performansından etkilenme riskini azaltırsınız.
Coğrafi Çeşitlendirme: Yatırımları farklı ülkelerde veya bölgelerde yaparak, yerel ekonomik olayların portföy üzerindeki etkisini minimize edersiniz.
Varlık Sınıfı Çeşitlendirmesi: Hisse senetleri, tahviller, emlak ve hatta kripto paralar gibi farklı varlık sınıflarına yatırım yaparak riskinizi dağıtırsınız.

Kişisel Finans Soruları:
Q:Sürdürülebilir yatırımlar ve yeşil enerji hisseleri hakkında ne düşünüyorsunuz, hangilerini önerirsiniz?
A:Sürdürülebilir yatırımlar, özellikle yeşil enerji ve çevre dostu teknolojiler, geleceğin trendleri arasında yer alıyor. Şirketlerin çevresel, sosyal ve yönetişim (ESG) kriterlerine olan bağlılıkları, yatırımcılar için önemli bir değerlendirme faktörüdür.
Q:Döviz kurundaki ani değişiklikler portföyümü nasıl etkiler ve bu durumda ne yapmalıyım?
A:Döviz kurlarındaki ani değişiklikler, özellikle ithalat ve ihracat yapan şirketlerin maliyetlerini ve kârlılığını etkileyebilir. Bu tür durumlarda döviz riskini hedge etmek için finansal araçlar kullanılabilir.





{{agent_scratchpad}}
"""

prompt = ChatPromptTemplate.from_template(template)


In [9]:
agent = create_openai_functions_agent(model,tools,prompt)
main_agent = AgentExecutor(agent=agent,tools=tools,memory=memory,verbose=True)

In [63]:
memory.clear()

In [74]:
main_agent.invoke({
        "input":"Türkiye Petrol Rafinerileri A.Ş. son çeyrek bilançosu nasıl geldi?"
    }
)



> Entering new AgentExecutor chain...

Invoking: `toolkit` with `Türkiye Petrol Rafinerileri A.Ş. son çeyrek bilançosu`


[PolygonAggregates(api_wrapper=PolygonAPIWrapper(polygon_api_key='EbIjp0WJIBUt2SQoE6bPA_sa3x2pREpe')), PolygonLastQuote(api_wrapper=PolygonAPIWrapper(polygon_api_key='EbIjp0WJIBUt2SQoE6bPA_sa3x2pREpe')), PolygonTickerNews(api_wrapper=PolygonAPIWrapper(polygon_api_key='EbIjp0WJIBUt2SQoE6bPA_sa3x2pREpe')), PolygonFinancials(api_wrapper=PolygonAPIWrapper(polygon_api_key='EbIjp0WJIBUt2SQoE6bPA_sa3x2pREpe'))]
Invoking: `tavily_search` with `Türkiye Petrol Rafinerileri A.Ş. son çeyrek bilançosu`


[{'url': 'https://www.gcmyatirim.com.tr/arastirma-analiz/ozel-raporlar/2024-05-23-tupras-turkiye-petrol-rafinerileri-tuprs-2024-1-ceyrek-bilanco-analizi', 'content': 'Araştırma Analiz Özel Rapor Tüpraş Türkiye Petrol Rafinerileri (TUPRS) 2024 1. Çeyrek Bilanço Analizi. Tüpraş Türkiye Petrol Rafinerileri (TUPRS) 2024 1. Çeyrek Bilanço Analizi. ... Şirket son kapanışa göre 323,8

{'input': 'Türkiye Petrol Rafinerileri A.Ş. son çeyrek bilançosu nasıl geldi?',
 'history': "Human: TUPRS hissesinin pazar büyüklüğü nedir?\nAI: Tüpraş (TUPRS) hissesi için pazar büyüklüğü ile ilgili net bir bilgi bulamadım. Ancak, Tüpraş, Türkiye'nin en büyük petrol rafinerisi ve enerji şirketlerinden biridir. Şirket, ham petrol ve petrol ürünlerinin rafinasyonunu yapmaktadır ve geniş bir ürün yelpazesine sahiptir.\n\nTüpraş'ın pazar büyüklüğünü etkileyen faktörler arasında, piyasa değeri, hisselerin işlem hacmi ve sektördeki genel trendler yer almaktadır. Şirketin finansal durumu ve piyasa performansı hakkında daha fazla bilgi için aşağıdaki kaynaklara göz atabilirsiniz:\n\n1. [Tüpraş Resmi Web Sitesi](https://www.tupras.com.tr/en/)\n2. [Google Finance Tüpraş Sayfası](https://www.google.com/finance/quote/TUPRS:IST)\n3. [Wikipedia - Tüpraş](https://en.wikipedia.org/wiki/Tüpraş)\n\nEğer daha fazla bilgiye ihtiyacınız varsa veya başka bir konuda yardımcı olmamı isterseniz, lütfen belirt

In [75]:
main_agent.invoke({
        "input":"BIST100 de işlem gören enerji sektöründen 3 hisse senedi seçip bana bir örnek portföy oluşturur musun? Bütçem 35.000 TL. Risk toleransım orta düzeyde."
    }
)



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `BIST100 enerji sektörü hisse senetleri`


[{'url': 'https://tr.investing.com/indices/ise-electricity-components', 'content': 'BIST Elektrik Bileşenler. Bu sayfa gerçek zamanlı BIST Elektrik Endeksi Bileşenleri yayın akışını içerir. Tabloda her bileşen için hisse senedinin adını, son fiyatını, günlük yüksek, düşük ve değişim değerlerini bulabilirsiniz. Hac. Bu sayfa gerçek zamanlı BIST Elektrik Endeksi Bileşenleri yayın akışını içerir..'}, {'url': 'https://www.gcmyatirim.com.tr/borsa/endeks/bist-elektrik-xelkt', 'content': "BIST Enerji hisseleri ve endeksine dair veriler, analizler ve piyasa gelişmeleri GCM Yatırım'da. XELKT hisselerinin grafikleri ve uzman yorumları için tıklayın. Lisanslı ve Geniş Yetkili Aracı Kurum - Lisans No: G-039 (398)"}, {'url': 'https://www.bloomberght.com/borsa/hisseler/bist-100-endeksi', 'content': 'BIST 100 endeksine ait Güncel hisse senedi fiyatları ve BIST hisse günlük verilerini Blo

{'input': 'BIST100 de işlem gören enerji sektöründen 3 hisse senedi seçip bana bir örnek portföy oluşturur musun? Bütçem 35.000 TL. Risk toleransım orta düzeyde.',
 'history': "Human: TUPRS hissesinin pazar büyüklüğü nedir?\nAI: Tüpraş (TUPRS) hissesi için pazar büyüklüğü ile ilgili net bir bilgi bulamadım. Ancak, Tüpraş, Türkiye'nin en büyük petrol rafinerisi ve enerji şirketlerinden biridir. Şirket, ham petrol ve petrol ürünlerinin rafinasyonunu yapmaktadır ve geniş bir ürün yelpazesine sahiptir.\n\nTüpraş'ın pazar büyüklüğünü etkileyen faktörler arasında, piyasa değeri, hisselerin işlem hacmi ve sektördeki genel trendler yer almaktadır. Şirketin finansal durumu ve piyasa performansı hakkında daha fazla bilgi için aşağıdaki kaynaklara göz atabilirsiniz:\n\n1. [Tüpraş Resmi Web Sitesi](https://www.tupras.com.tr/en/)\n2. [Google Finance Tüpraş Sayfası](https://www.google.com/finance/quote/TUPRS:IST)\n3. [Wikipedia - Tüpraş](https://en.wikipedia.org/wiki/Tüpraş)\n\nEğer daha fazla bilgi

In [47]:
main_agent.invoke({
        "input":"Evlilik için birikim yapmak istiyorum. Tavsiyelerin nelerdir?"
    }
)



> Entering new AgentExecutor chain...
Evlilik için birikim yapmak istiyorsanız, dikkate almanız gereken birkaç önemli nokta var:

1. **Hedef Belirleme**: İlk olarak, ne kadar birikim yapmak istediğinizi belirleyin. Evlilik masrafları, düğün, balayı ve diğer harcamalar için toplam bir bütçe oluşturun.

2. **Birikim Planı**: Hedeflediğiniz miktarı ve zaman dilimini göz önünde bulundurarak, her ay ne kadar birikim yapmanız gerektiğini hesaplayın. Bu, düzenli bir tasarruf planı oluşturmanıza yardımcı olacaktır.

3. **Yatırım Araçları**: Birikimlerinizi değerlendirmek için farklı yatırım araçlarını düşünebilirsiniz. Örneğin:
   - **Tasarruf Hesapları**: Düşük riskli ve likit bir seçenek.
   - **Hisse Senetleri ve Fonlar**: Uzun vadede daha yüksek getiri potansiyeli sunar, ancak risklidir.
   - **Tahviller**: Daha stabil bir getiri sağlar.
   - **Bireysel Emeklilik Fonları**: Uzun vadeli birikim için uygun olabilir.

4. **Acil Durum Fonu**: Evlilik öncesi beklenmedik harcamalar için bir ac

{'input': 'Evlilik için birikim yapmak istiyorum. Tavsiyelerin nelerdir?',
 'history': 'Human: TUPRS hissesinin pazar büyüklüğü nedir?\nAI: Türkiye Petrol Rafinerileri A.Ş. (TUPRS) hissesinin pazar büyüklüğü (market capitalization) şu anda yaklaşık 9.82 milyar USD olarak belirlenmiştir. Bu değer, şirketin halka açık paylarının toplam değerini ifade etmektedir ve Türkiye Petrol Rafinerileri, bu pazar büyüklüğü ile dünya genelinde 1688. en değerli şirket konumundadır. \n\nDaha fazla bilgi için şu kaynakları inceleyebilirsiniz:\n- [Yahoo Finance](https://finance.yahoo.com/quote/TUPRS.IS/)\n- [Google Finance](https://www.google.com/finance/quote/TUPRS:IST)\n- [Companies Market Cap](https://companiesmarketcap.com/turkiye-petrol-rafinerileri/marketcap/)\n- [Bloomberg](https://www.bloomberg.com/quote/TUPRS:TI)\n- [Wall Street Journal](https://www.wsj.com/market-data/quotes/tr/tuprse)\n\nUnutmayın ki bu bilgiler yatırım tavsiyesi değildir.',
 'output': 'Evlilik için birikim yapmak istiyorsanız

In [17]:
main_agent.invoke({
        "input":"ASELSAN hakkında bilgi verir misin?"
    }
)



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `ASELSAN`


[{'url': 'https://www.aselsan.com/tr', 'content': "ASELSAN, Türkiye'nin en büyük savunma ve sivil teknolojileri üreticisidir. Savunma alanında yüksek teknoloji ürün ve sistemler, sivil alanlarda sürdürülebilirliği ve inovasyonu öne çıkaran ASELSAN, haberler ve duyurularında son gelişmelerini paylaşmaktadır."}, {'url': 'https://en.wikipedia.org/wiki/Aselsan', 'content': 'Aselsan (Turkish: Aselsan, acronym: Askerî Elektronik Sanayi, Military Electronic Industries), Aselsan A.Ş., is a Turkish defense corporation headquartered in Ankara, Turkey. Its main operating area is research, development and manufacture of advanced military products for air, land and maritime forces.'}, {'url': 'https://www.defensenews.com/industry/2023/07/18/turkish-aerospace-aselsan-ink-2b-in-deals-to-upgrade-f-16-jets/', 'content': 'â€œThis deal has two implications: One, it is part of Turkish efforts to balance Greeceâ€™s increasi

{'input': 'ASELSAN hakkında bilgi verir misin?',
 'history': "Human: Merhaba, borsa hakkında bilgi verebilir misin?\nAI: Merhaba! Tabii ki, borsa, hisse senetleri, tahviller ve diğer menkul kıymetlerin alınıp satıldığı bir piyasa yeridir. Yatırımcılar, şirketlerin hisse senetlerini satın alarak şirketlere ortak olurlar ve şirketlerin kârlarından pay alabilirler.\nHuman: Borsada yatırım yapmanın riskleri nelerdir?\nAI: Borsada yatırım yapmanın bazı riskleri vardır. Bunlar arasında piyasa riski, likidite riski ve şirket riski bulunur. Piyasa riski, genel piyasa dalgalanmalarından kaynaklanan kayıpları ifade eder. Likidite riski, yatırımcıların varlıklarını hızlı bir şekilde nakde çevirememesinden kaynaklanır. Şirket riski ise, belirli bir şirketin performansına bağlı risklerdir.\nHuman: Borsa endeksleri nedir ve nasıl çalışır?\nAI: Borsa endeksleri, belirli bir grup hisse senedinin performansını ölçen göstergelerdir. Örneğin, S&P 500, Amerika Birleşik Devletleri'ndeki en büyük 500 şirket

In [23]:
main_agent.invoke({
        "input":"TUPRS son fiyatı nedir?"
    }
)



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `TUPRS son fiyat`


[{'url': 'https://tr.tradingview.com/symbols/BIST-TUPRS/', 'content': "5. TUPRS-TÜPRAŞ Dolar Bazlı Haftalık Grafik 🔸%30'un üstünde getiri sağlayan son yükselişin ardından düzeltme hareketine başlayan Tüpraş, açıkladığı kötü bilanço sonrası hareketi hızlandı. Beklentim 5.07-5.1 $ civarında dengelenmesi. Yani 165 TL civarı. Tek çekincem son yükselen trend desteğini kırmış olması."}, {'url': 'https://tr.investing.com/equities/tupras', 'content': 'Türkiye Petrol Rafinerileri A.Ş. hissesi ile ilgili grafiklere, analizlere, Tüpraş Hisse Fiyatları (TUPRS) raporlarına ve daha fazlasına ulaşın. 50% İndirim. 50% İndirim. Giriş Yapın ... Son Fiyat Fark % Hac. Milpa. 119,900 +10,00%: 1,64M.'}, {'url': 'https://bigpara.hurriyet.com.tr/borsa/hisse-fiyatlari/tuprs-tupras-detay/', 'content': "56,91. RSI. 44,89. CCI. -13,28. TEKNİK ANALİZ. TUPRS hissesi 10.7.2024 tarihinde günü 168,00 TL'den ve %-0,47 deger kayb

{'input': 'TUPRS son fiyatı nedir?',
 'output': 'TÜPRAŞ (TUPRS) son fiyatı 168 TL civarındadır. Daha fazla bilgi için [buraya](https://bigpara.hurriyet.com.tr/borsa/hisse-fiyatlari/tuprs-tupras-detay/) bakabilirsiniz.'}

In [21]:
main_agent.invoke({
        "input":"Bana Bitcoin'in fiyatını güncel olarak verir misin?"
    }
)



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `Bitcoin fiyatı`


[{'url': 'https://www.binance.com/tr/price/bitcoin', 'content': "Bitcoin, piyasadaki en popüler kripto para birimi olup, fiyatı, arzı ve hacmi ile dünyanın ekonomisinde önemli bir rol oynar. Binance'da Bitcoin fiyatını güncel olarak takip edebilir, grafikler, karşılaştırmalar ve fiyat endeksi ile döviz kurlarına dönüştürebilirsiniz."}, {'url': 'https://tr.investing.com/crypto/bitcoin', 'content': "Bitcoin, kripto para piyasasının lideri olan BTC, Amerikan doları karşısında güncel fiyatı 64.385,5 dolar olarak gösteriyor. Investing.com'da Bitcoin'ın son gelişmeleri, teknik analizleri, yatırımcı yorumları ve daha fazlasını bulabilirsiniz."}, {'url': 'https://coinmarketcap.com/currencies/bitcoin/', 'content': 'Bitcoin analytics\nPeople also watch\n1.36%\n4.20%\n0.39%\n23.09%\n5.69%\n1.34%\n2.27%\n2.37%\n0.52%\n5.12%\nTrending\n0.66%\n28.91%\n58.41%\n10.22%\n3.67%\nBitcoin Price Live Data\nThe live Bi

{'input': "Bana Bitcoin'in fiyatını güncel olarak verir misin?",
 'output': "Bitcoin'in güncel fiyatı farklı kaynaklara göre değişiklik gösterebilir. Şu anda bazı kaynaklara göre Bitcoin'in fiyatı şu şekildedir:\n\n- Binance: [Bitcoin Fiyatı](https://www.binance.com/tr/price/bitcoin)\n- Investing.com: Bitcoin'in güncel fiyatı 64.385,5 dolar olarak gösteriliyor. [Detaylar için tıklayın](https://tr.investing.com/crypto/bitcoin)\n- CoinMarketCap: Bitcoin'in güncel fiyatı 37,246.73 USD ve 24 saatlik işlem hacmi 11,983,504,250 USD. [Detaylar için tıklayın](https://coinmarketcap.com/currencies/bitcoin/)\n- CoinDesk: Bitcoin'in fiyatı ve güncel gelişmeler h"}

In [22]:
main_agent.invoke({
        "input":"Bana Türkiye'de 2023 yılında en çok büyüyen sektörü söyler misin?"
    }
)



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `Türkiye'de 2023 yılında en çok büyüyen sektör`


[{'url': 'https://www.hurriyet.com.tr/ekonomi/buyume-rakamlari-aciklandi-42413330', 'content': 'Türkiye ekonomisi 2023 yılı 4. çeyreğinde yüzde 4 büyüdü. 2023 yılında ise Türkiye ekonomisi yüzde 4,5 büyüdü. Türkiye, AB ülkeleri arasında en çok büyüyen...'}, {'url': 'https://bigpara.hurriyet.com.tr/haberler/genel-haberler/turkiyenin-2023-yili-buyume-rakamlari-aciklandi_ID1479725/', 'content': "Türkiye İstatistik Kurumu (TÜİK) Türkiye'nin 2023 büyüme rakamlarını açıkladı. 2023 yılında gayri safi yurt içi hasıla (GSYH) yüzde 4,5 büyüdü. Türkiye, AB ülkeleri arasında en çok büyüyen ülke oldu. GSYH böylece ilk kez 2023 itibarıyla 1 trilyon dolarlık büyüklüğü aşarak 1 trilyon 118 milyar 593 milyon ..."}, {'url': 'https://www.bloomberght.com/turkiye-ekonomisi-2023-te-yuzde-4-5-buyudu-2348413', 'content': "Türkiye ekonomisi 2023 yılında yüzde 4,5 büyüme kaydetti. Ekonomi 20

{'input': "Bana Türkiye'de 2023 yılında en çok büyüyen sektörü söyler misin?",
 'output': "Türkiye'de 2023 yılında en hızlı büyüyen sektör yüzde 9 büyüme ile finans ve sigorta faaliyetleri sektörü olmuştur. Bu sektörün yanı sıra inşaat sektörü de yüzde 7,8 büyüme göstermiştir. Sanayide ise yüzde 0,8'lik bir büyüme kaydedilmiştir. Tarım sektörü ise yüzde 0,2 daralmıştır. Bu verilere göre finans ve sigorta faaliyetleri sektörü Türkiye'de 2023 yılında en çok büyüyen sektör olmuştur."}

In [24]:
main_agent.invoke({
        "input":"Yüklü miktarda param var ama bunları enflasyona karşı korumak istiyorum. Altın mantıklı bir yatırım aracı mıdır?"
    }
)



> Entering new AgentExecutor chain...
Altın genellikle enflasyona karşı bir koruma aracı olarak görülmektedir. Enflasyon arttığında genellikle altın fiyatları da artar, bu nedenle yatırımcılar enflasyona karşı korunmak için altına yatırım yapabilirler. Ancak altın yatırımı da risk içermektedir ve fiyatları piyasa koşullarına göre değişkenlik gösterebilir.

Yatırım yapmadan önce dikkatlice araştırma yapmanız ve riskleri değerlendirmeniz önemlidir. Ayrıca yatırım tavsiyesi almak için finansal danışmanlardan destek almanız da faydalı olabilir. Eğer daha fazla bilgi

> Finished chain.


{'input': 'Yüklü miktarda param var ama bunları enflasyona karşı korumak istiyorum. Altın mantıklı bir yatırım aracı mıdır?',
 'history': '',
 'output': 'Altın genellikle enflasyona karşı bir koruma aracı olarak görülmektedir. Enflasyon arttığında genellikle altın fiyatları da artar, bu nedenle yatırımcılar enflasyona karşı korunmak için altına yatırım yapabilirler. Ancak altın yatırımı da risk içermektedir ve fiyatları piyasa koşullarına göre değişkenlik gösterebilir.\n\nYatırım yapmadan önce dikkatlice araştırma yapmanız ve riskleri değerlendirmeniz önemlidir. Ayrıca yatırım tavsiyesi almak için finansal danışmanlardan destek almanız da faydalı olabilir. Eğer daha fazla bilgi'}

In [29]:
main_agent.invoke({
        "input":"Peki bana Temmuz ayı fiyatını söyler misin?"
    }
)



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `Temmuz ayı altın fiyatı`


[{'url': 'https://www.bloomberght.com/gram-altinda-son-durum-ne-11-temmuz-2024-persembe-altin-fiyatlari-pkh1-2356268', 'content': 'Gözde yatırım araçlarından altın da hareketlilik devam ediyor. Altının ons fiyatı bugün saat 10:00 itibariyle 2,382 dolardan işlem görüyor. Düne göre değişim oranı %0.45. 11 Temmuz 2024 güncel gram altın fiyatı düne göre yüzde 0.67 artış gösterdi. Güncel altın fiyatları (11 Temmuz 2024 Perşembe)'}, {'url': 'https://www.ntv.com.tr/ntvpara/altin-fiyatlari-ne-kadar-gram-altin-kac-lira-12-temmuz-2024-guncel-altin-kuru-fiyatlari,vG9U8gJv5EKD52wqyedPcA', 'content': "Altın fiyatları, haftanın son işlem gününde negatif seyretti. Ons altın, bugün yüzde 0,27 düşüşle 2.409 dolar civarında işlem gördü. Gram altın, 2.560 Türk Lirası'ndan alıcı ..."}, {'url': 'https://www.hesapkurdu.com/altin', 'content': "Altın fiyatlarına anında ulaşmak istiyorsanız, Hesapkurdu.com'un al

{'input': 'Peki bana Temmuz ayı fiyatını söyler misin?',
 'history': 'Human: Yüklü miktarda param var ama bunları enflasyona karşı korumak istiyorum. Altın mantıklı bir yatırım aracı mıdır?\nAI: Altın genellikle enflasyona karşı bir koruma aracı olarak görülmektedir. Enflasyon arttığında genellikle altın fiyatları da artar, bu nedenle yatırımcılar enflasyona karşı korunmak için altına yatırım yapabilirler. Ancak altın yatırımı da risk içermektedir ve fiyatları piyasa koşullarına göre değişkenlik gösterebilir.\n\nYatırım yapmadan önce dikkatlice araştırma yapmanız ve riskleri değerlendirmeniz önemlidir. Ayrıca yatırım tavsiyesi almak için finansal danışmanlardan destek almanız da faydalı olabilir. Eğer daha fazla bilgi\nHuman: Peki bana 2024 yılı Nisan ayı fiyatını söyler misin?\nAI: 2024 yılı Nisan ayında TÜFE yüzde 3,18 oranında artmıştır. Nisan ayındaki artışla birlikte yıllık enflasyon bir önceki aya kıyasla 1,30 puan artarak yüzde 69,80 düzeyinde gerçekleşmiştir. Yüzde 2,78 oranınd

In [18]:
main_agent.invoke({
        "input":"Elimde 1000000 TL var bunu nasıl derğelendirebilirim?"
    }
)



> Entering new AgentExecutor chain...
Elinizdeki 1.000.000 TL'yi değerlendirmek için birkaç farklı seçenek bulunmaktadır. İşte bazı öneriler:

1. **Borsa Yatırımı**: Hisse senetleri alarak borsa piyasasında yatırım yapabilirsiniz. Ancak, borsa riskli bir yatırım aracı olduğu için iyi bir araştırma yapmanız ve piyasa trendlerini takip etmeniz önemlidir.

2. **Vadeli Mevduat**: Bankalarda vadeli mevduat hesabı açarak belirli bir süre için paranızı değerlendirebilirsiniz. Bu yöntem genellikle daha düşük risk taşır ancak getiri oranları da borsa gibi yüksek olmayabilir.

3. **Gayrimenkul Yatırımı**: Emlak alarak uzun vadeli bir yatırım yapabilirsiniz. Gayrimenkul, değer kazanma potansiyeli olan bir yatırım aracı olabilir.

4. **Yatırım Fonları**: Çeşitli yatırım fonlarına yatırım yaparak profesyonel yöneticiler tarafından yönetilen bir portföy oluşturabilirsiniz.

5. **Kripto Para**: Kripto paralara yatırım yapmayı düşünebilirsiniz. Ancak, bu alandaki riskler oldukça yüksektir ve dikkatl

{'input': 'Elimde 1000000 TL var bunu nasıl derğelendirebilirim?',
 'history': "Human: Merhaba, borsa hakkında bilgi verebilir misin?\nAI: Merhaba! Tabii ki, borsa, hisse senetleri, tahviller ve diğer menkul kıymetlerin alınıp satıldığı bir piyasa yeridir. Yatırımcılar, şirketlerin hisse senetlerini satın alarak şirketlere ortak olurlar ve şirketlerin kârlarından pay alabilirler.\nHuman: Borsada yatırım yapmanın riskleri nelerdir?\nAI: Borsada yatırım yapmanın bazı riskleri vardır. Bunlar arasında piyasa riski, likidite riski ve şirket riski bulunur. Piyasa riski, genel piyasa dalgalanmalarından kaynaklanan kayıpları ifade eder. Likidite riski, yatırımcıların varlıklarını hızlı bir şekilde nakde çevirememesinden kaynaklanır. Şirket riski ise, belirli bir şirketin performansına bağlı risklerdir.\nHuman: Borsa endeksleri nedir ve nasıl çalışır?\nAI: Borsa endeksleri, belirli bir grup hisse senedinin performansını ölçen göstergelerdir. Örneğin, S&P 500, Amerika Birleşik Devletleri'ndeki e

In [30]:
main_agent.invoke({
        "input":"2024 yılında altın mı alınmalı bono mu?"
    }
)



> Entering new AgentExecutor chain...
Öngörülen ekonomik koşullar ve kişisel yatırım hedeflerinize bağlı olarak, altın ve bono gibi yatırım araçları arasında seçim yapabilirsiniz. Altın genellikle ekonomik belirsizlik dönemlerinde güvenli liman olarak tercih edilirken, bonolar ise düşük riskli ve sabit getirili yatırım araçlarıdır.

2024 yılında altın mı alınmalı yoksa bono mu tercih edilmeli konusunda kesin bir öneri veremem çünkü yatırım tercihleri kişiden kişiye değişebilir. Ancak, genel olarak altın fiyatları ekonomik belirsizliklerde yükselme eğ

> Finished chain.


{'input': '2024 yılında altın mı alınmalı bono mu?',
 'history': "Human: ASELSAN hakkında bilgi verir misin?\nAI: ASELSAN, Ankara, Türkiye merkezli bir Türk savunma şirketidir. Ana faaliyet alanı, ileri askeri teknoloji ürünlerinin araştırılması, geliştirilmesi ve üretilmesidir. ASELSAN, Türk Silahlı Kuvvetleri'nin iletişim ihtiyaçlarını karşılamak amacıyla 1975 yılında kurulan Türk Silahlı Kuvvetleri Vakfı şirketidir. ASELSAN, Türkiye'nin en büyük savunma şirketi olan Aselsan ve özel bir tersane olan Sefine arasındaki ortaklıkla geliştirilen Marlin SIDA'yı geliştirmiştir. Aselsan Elektronik Sanayi Ve Ticaret AS şirketinin his\nHuman: TUPRAS 2023 yılında ne kadar kar etti?\nAI: Malesef, TUPRAS'ın 2023 yılındaki net karı hakkında net bir bilgi bulunmamaktadır. Ancak, hisse senedi fiyatları ve hareketleri hakkında bilgi verilebilir. Eğer başka bir konuda yardımcı olabileceğim bir şey varsa lütfen belirtin.\nHuman: TUPRAS 2023 yılında ne kadar kar etti?\nAI: Malesef, TUPRAS'ın 2023 yılınd

In [10]:
main_agent.invoke({
        "input":"TUPRAS ne şirketidir?"
    }
)



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `TUPRAS`


[{'url': 'https://www.tupras.com.tr/en/', 'content': 'Tüpraş is transforming its business for a carbon-neutral world by 2050, producing sustainable energy types and using cutting-edge technology. Learn about its refineries, subsidiaries, innovation, culture, and agenda.'}, {'url': 'https://en.wikipedia.org/wiki/Tüpraş', 'content': "Introduction. Tüpraş operates four oil refineries, three of which process imported crude. The 11 million tonnes a year Izmit refinery at Izmit in north west Turkey and the 11 million tonnes/yr Izmir refinery at Aliaga on Turkey's central Aegean coast process crude from global markets delivered by tanker while the 5 million tonnes/yr Kirikkale ..."}, {'url': 'https://www.tupras.com.tr/en/about-us', 'content': 'Tüpraş is a carbon neutral energy company that aims to transform for a sustainable world by 2050. It operates four main business lines: sustainable refining, biofuels, ze

{'input': 'TUPRAS ne şirketidir?',
 'history': "Human: Merhaba, borsa hakkında bilgi verebilir misin?\nAI: Merhaba! Tabii ki, borsa, hisse senetleri, tahviller ve diğer menkul kıymetlerin alınıp satıldığı bir piyasa yeridir. Yatırımcılar, şirketlerin hisse senetlerini satın alarak şirketlere ortak olurlar ve şirketlerin kârlarından pay alabilirler.\nHuman: Borsada yatırım yapmanın riskleri nelerdir?\nAI: Borsada yatırım yapmanın bazı riskleri vardır. Bunlar arasında piyasa riski, likidite riski ve şirket riski bulunur. Piyasa riski, genel piyasa dalgalanmalarından kaynaklanan kayıpları ifade eder. Likidite riski, yatırımcıların varlıklarını hızlı bir şekilde nakde çevirememesinden kaynaklanır. Şirket riski ise, belirli bir şirketin performansına bağlı risklerdir.\nHuman: Borsa endeksleri nedir ve nasıl çalışır?\nAI: Borsa endeksleri, belirli bir grup hisse senedinin performansını ölçen göstergelerdir. Örneğin, S&P 500, Amerika Birleşik Devletleri'ndeki en büyük 500 şirketin hisse senet

In [11]:
main_agent.invoke({
        "input":"Şuanda Türkiyedeki mevcut işsizlik oranı nedir?"
    }
)



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `Türkiye mevcut işsizlik oranı`


[{'url': 'https://tr.tradingeconomics.com/turkey/unemployment-rate', 'content': 'Akım değerleri, tarihsel veriler, tahminler, istatistikler, grafikler ve ekonomik takvim - Türkiye - İşsizlik Oranı. 2005-2024 Veri | 2025-2026 Tahmin.'}, {'url': 'https://www.sbb.gov.tr/istihdam/', 'content': 'Ana Sayfa « Ekonomik Veriler « İstihdam 10 Temmuz 2024 tarihinde açıklanan işgücü istatistiklerine göre işsizlik oranı 2024 yılı Mayıs ayında yüzde 7,7 seviyesinde gerçekleşti. İstihdam edilenlerin sayısı 33 milyon 233 bin kişi, istihdam oranı ise yüzde 50,5 oldu. Mevsim etkilerinden arındırılmış veriler dikkate alındığında; İstihdam edilenlerin sayısı 32 ...'}, {'url': 'https://data.tuik.gov.tr/Kategori/GetKategori?p=istihdam-issizlik-ve-ucret-108&dil=1', 'content': 'Coğrafi İstatistik Portalı. Ulusal Veri Sayfası. TürkçeEnglish. İstihdam, İşsizlik ve Ücret. 402. Alt Kategoriler. İşgücü İstatis

{'input': 'Şuanda Türkiyedeki mevcut işsizlik oranı nedir?',
 'history': "Human: Merhaba, borsa hakkında bilgi verebilir misin?\nAI: Merhaba! Tabii ki, borsa, hisse senetleri, tahviller ve diğer menkul kıymetlerin alınıp satıldığı bir piyasa yeridir. Yatırımcılar, şirketlerin hisse senetlerini satın alarak şirketlere ortak olurlar ve şirketlerin kârlarından pay alabilirler.\nHuman: Borsada yatırım yapmanın riskleri nelerdir?\nAI: Borsada yatırım yapmanın bazı riskleri vardır. Bunlar arasında piyasa riski, likidite riski ve şirket riski bulunur. Piyasa riski, genel piyasa dalgalanmalarından kaynaklanan kayıpları ifade eder. Likidite riski, yatırımcıların varlıklarını hızlı bir şekilde nakde çevirememesinden kaynaklanır. Şirket riski ise, belirli bir şirketin performansına bağlı risklerdir.\nHuman: Borsa endeksleri nedir ve nasıl çalışır?\nAI: Borsa endeksleri, belirli bir grup hisse senedinin performansını ölçen göstergelerdir. Örneğin, S&P 500, Amerika Birleşik Devletleri'ndeki en büyü